# Occupancy detection

### A multivariate time series example

In this tutorial, you will learn how to apply getML to multivariate time series. It also demonstrates how to use getML's [high-level interface for hyperparameter tuning](https://docs.getml.com/latest/user_guide/hyperopt/hyperopt.html#tuning-routines).

Summary:

- Prediction type: __Binary classification__
- Domain: __Energy__
- Prediction target: __Room occupancy__
- Source data: __1 table, 32k rows__
- Population size: __32k__

_Author: Dr. Johannes King_

# Background

Our use case is a public domain data set for predicting room occupancy from sensor data. The results achieved using getML outperform all published results on this data set. Note that this is not only a neat use case for machine learning algorithms, but a real-world application with tangible consequences: If room occupancy is known with sufficient certainty, it can be applied to the control systems of a building. Such as system can reduce the energy consumption by [up to 50 %](https://ieeexplore.ieee.org/document/7566062).


### Introduction to occupancy prediction

Usually, getML is considered to be a tool for feature engineering and machine learning on relational data sets. How can we apply it to (multivariate) time series?

The key is a _self-join_. Instead of creating features by merging and aggregating peripheral tables in a relational data model, for a time-series, we perform the same operations on the population table itself. This results in features like these:

* Aggregations over time, such as the average value of some column for the last 3 days.

* Seasonal effects, such as today is a Wednesday, so let's get the average value for the last four Wednesdays.

* Lag variables, such as get the value of some column from two hours ago.

Using getML's algorithms for relational learning, we can extract all of these features automatically. Having created a flat table of such features, we can then apply state-of-the-art machine learning algorithms, like xgboost. As you will see in this example, this performs better than traditional time series analysis.

The present analysis is based on a public domain time series dataset. It is available in the [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+). The challenge is straightforward: We want to predict whether an office room is occupied at a given moment in time using sensor data. The data is measured about once a minute. Ground-truth occupancy was obtained from time-stamped pictures. The available columns are

* Date, year-month-day hour:minute:second
* Temperature, in Celsius
* Relative Humidity, %
* Light, in Lux
* CO2, in ppm
* Humidity Ratio, Derived quantity from temperature and relative humidity, in kgwater-vapor/kg-air
* Occupancy, 0 or 1, 0 for not occupied, 1 for occupied status

As a reference and benchmark, we use [this paper](https://www.sciencedirect.com/science/article/abs/pii/S0378778815304357):

> Accurate occupancy detection of an office room from light, temperature, humidity and CO2 measurements using statistical learning models. Luis M. Candanedo, Veronique Feldheim. Energy and Buildings. Volume 112, 15 January 2016, Pages 28-39.

The authors apply various artifical neural networks algorithms to the data set at hand and achieved accuracies between 80.324% (batch back algorithm) and 99.061% (limited memory quasi-Newton algorithm).

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set-up your session:

In [24]:
import os
from urllib import request

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image, Markdown

%matplotlib inline

import getml

getml.engine.launch()
getml.engine.set_project("occupancy")

getML engine is already running.
Loading pipelines... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Connected to project 'occupancy'


## 1. Loading data


The data set can be downloaded directly from GitHub. It is conveniently separated into a training, a validation and a testing set. This allows us to directly benchmark our results against the results of the original paper later.

In [25]:
(
    population_train,
    population_test,
    population_validation,
) = getml.datasets.load_occupancy()


Loading population_train...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Loading population_test...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Loading population_validation...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          


In [26]:
population, split = getml.data.split.concat(
    "population",
    train=population_train,
    validation=population_validation,
    test=population_test,
)

The training set looks like this:

In [27]:
population

name,date,Occupancy,Temperature,Humidity,Light,CO2,HumidityRatio
role,time_stamp,target,numerical,numerical,numerical,numerical,numerical
unit,time stamp,,,,,,
0,2015-02-04 17:51:00,1,23.18,27.272,426,721.25,0.004793
1,2015-02-04 17:51:59,1,23.15,27.2675,429.5,714,0.004783
2,2015-02-04 17:53:00,1,23.15,27.245,426,713.5,0.004779
3,2015-02-04 17:54:00,1,23.15,27.2,426,708.25,0.004772
4,2015-02-04 17:55:00,1,23.1,27.2,426,704.5,0.004757
,...,...,...,...,...,...,...
20555,2015-02-18 09:15:00,1,20.815,27.7175,429.75,1505.25,0.004213
20556,2015-02-18 09:16:00,1,20.865,27.745,423.5,1514.5,0.00423


In [28]:
split

,
0,train
1,train
2,train
3,train
4,train
,...


We also assign roles to each column. To learn more about what roles do and why we need them, check out the [official documentation](https://docs.getml.com/latest/user_guide/annotating_data/annotating_data.html).

In [29]:
population.set_role(["Occupancy"], getml.data.roles.target)
population.set_role(
    ["Temperature", "Humidity", "Light", "CO2", "HumidityRatio"],
    getml.data.roles.numerical,
)
population.set_role(["date"], getml.data.roles.time_stamp)

population

name,date,Occupancy,Temperature,Humidity,Light,CO2,HumidityRatio
role,time_stamp,target,numerical,numerical,numerical,numerical,numerical
unit,time stamp,,,,,,
0,2015-02-04 17:51:00,1,23.18,27.272,426,721.25,0.004793
1,2015-02-04 17:51:59,1,23.15,27.2675,429.5,714,0.004783
2,2015-02-04 17:53:00,1,23.15,27.245,426,713.5,0.004779
3,2015-02-04 17:54:00,1,23.15,27.2,426,708.25,0.004772
4,2015-02-04 17:55:00,1,23.1,27.2,426,704.5,0.004757
,...,...,...,...,...,...,...
20555,2015-02-18 09:15:00,1,20.815,27.7175,429.75,1505.25,0.004213
20556,2015-02-18 09:16:00,1,20.865,27.745,423.5,1514.5,0.00423


## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 getML Pipeline

We use a [Multirel](https://docs.getml.com/latest/api/getml.models.MultirelModel.html) for generating the features and a simple logistic regression for prediction.

We do not spend much effort on the hyperparameters and largely go with the default values. The only exception is that we add some regularization to the XGBoostClassifiers.

We choose to consider data within the last 15 minutes for creating our features.

In [30]:
# Our forecast horizon is 0.
# We do not predict the future, instead we infer
# the present state from current and past sensor data.
horizon = 0.0

# We do not allow the time series features
# to use target values from the past.
# (Otherwise, we would need the horizon to
# be greater than 0.0).
allow_lagged_targets = False

# We want our time series features to only use
# data from the last 15 minutes
memory = getml.data.time.minutes(15)

time_series = getml.data.TimeSeries(
    population=population,
    split=split,
    time_stamps="date",
    horizon=horizon,
    memory=memory,
    lagged_targets=allow_lagged_targets,
)

time_series

data model

  population:
    columns:
    - Temperature: numerical
    - Humidity: numerical
    - Light: numerical
    - CO2: numerical
    - HumidityRatio: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 900.0
      horizon: 0.0
      lagged_targets: False

  population:
    columns:
    - Temperature: numerical
    - Humidity: numerical
    - Light: numerical
    - CO2: numerical
    - HumidityRatio: numerical
    - ...


container

  population
      subset       name         rows   type
  0   test         population   9751   View
  1   train        population   8144   View
  2   validation   population   2665   View

  peripheral
      name          rows   type     
  0   population   20560   DataFrame

In [31]:
feature_learner = getml.feature_learning.Multirel(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
)

predictor = getml.predictors.LogisticRegression()

pipe = getml.pipeline.Pipeline(
    data_model=time_series.data_model,
    tags=["memory=15", "logistic regression"],
    feature_learners=[feature_learner],
    predictors=[predictor],
)

pipe

Pipeline(data_model='population',
         feature_learners=['Multirel'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='CrossEntropyLoss',
         peripheral=['population'],
         predictors=['LogisticRegression'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory=15', 'logistic regression'])

`.check(...)` will be automatically called by `.fit(...)`. But it is always a good idea to call `.check(...)` separately before fitting, so we still have time for some last-minute fixes.

In [32]:
pipe.check(time_series.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


### 2.2 Model training

We use a routine for automatic [hyperparameter optimization](https://docs.getml.com/latest/api_documentation/python/hyperopt.html) to find the best parameters for the predictor:

In [33]:
pipe = getml.hyperopt.tune_predictors(pipe, time_series)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Tuning logistic regression... 100% |██████████| [elapsed: 02:07, remaining: 00:00]          

Time taken: 0h:2m:7.291137

Building final pipeline...

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Multirel: Training features... 100% |██████████| [elapsed: 00:06, remaining: 00:00]          
Multirel: Building features... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Multirel: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          
LogisticRegression: Training as predictor... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          

Trained pipeline.
Time take

The logistic regression has one hyperparameter that can be optimized, namely its L2 regularization parameter. We fine-tune this parameter using the validation set. Note that the validation set is different from the testing set on which the final outcome will be evaluated.

### 2.3 Model evaluation

Let's see how well we did by scoring the model on the testing set.

In [34]:
pipe.score(time_series.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Multirel: Building features... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2023-08-01 00:01:01,train,Occupancy,0.9887,0.9948,0.5534
1,2023-08-01 00:01:02,validation,Occupancy,0.9786,0.9912,0.5275
2,2023-08-01 00:01:10,test,Occupancy,0.9934,0.9976,0.5534


In the [original paper](http://www.worldresearchlibrary.org/up_proc/pdf/568-148612088816-20.pdf), the authors tried several approaches. The best out-of-sample values of all the approaches they tried are the following:

* Accuracy (testing): 0.99061
* AUC (testing): 0.99574

Note that our results outperform the best approach from the original paper, both in terms of accuracy as well as AUC.
This demonstrates how powerful getML's relational learning approach is also a powerful tool for time series.

In [ ]:
# Refers to the data from the last time
# we called .score(...).
fpr, tpr = pipe.plots.roc_curve()

plt.subplots(figsize=(20, 10))

plt.plot(fpr, tpr, color="#6829c2")

plt.title("receiver operating characteristic (ROC)")
plt.grid(True)
plt.xlabel("false positive rate")
plt.ylabel("true positive rate")

plt.show()

In [ ]:
# Refers to the data from the last time
# we called .score(...).
recall, precision = pipe.plots.precision_recall_curve()

plt.subplots(figsize=(20, 10))

plt.plot(recall, precision, color="#6829c2")

plt.title("precision-recall curve")
plt.grid(True)
plt.xlabel("recall (true positive rate)")
plt.ylabel("precision")

plt.show()

In [ ]:
# Refers to the data from the last time
# we called .score(...).
proportion, lift = pipe.plots.lift_curve()

plt.subplots(figsize=(20, 10))

plt.plot(proportion, lift, color="#6829c2")

plt.title("lift curve")
plt.grid(True)
plt.xlabel("proportion")
plt.ylabel("lift")

plt.show()

### 2.4 Studying the features

It is always a good idea to study the features the relational learning algorithm has extracted. We can do so in the [feature view](https://docs.getml.com/latest/user_guide/getml_suite/monitor/models.html#feature-view) of the getML monitor. Open the monitor and select the models tab in the sidebar. You will see an overview over the trained pipelines. Select a pipeline to see the most essential summary plots.

If you want to document them inside your notebook, here is how you can do that:

In [ ]:
pipe.features

In [ ]:
names, correlations = pipe.features.correlations()

plt.subplots(figsize=(20, 10))

plt.bar(names, correlations, color="#6829c2")

plt.title("feature correlations")
plt.grid(True)
plt.xlabel("features")
plt.ylabel("correlations")
plt.xticks(rotation="vertical")

plt.show()

In [ ]:
names, importances = pipe.features.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color="#6829c2")

plt.title("feature importances")
plt.grid(True)
plt.xlabel("features")
plt.ylabel("importance")
plt.xticks(rotation="vertical")

plt.show()

The feature importance is calculated by XGBoost based on the improvement of the optimizing criterium at each split in the decision tree and is normalized to 100%.

We first look at the most important feature. The names returned by feature importances are already sorted, so we can just index them, like this:

In [ ]:
pipe.features.to_sql()[names[0]]

Let's check out the second most important feature.

In [ ]:
pipe.features.to_sql()[names[1]]

These two features demonstrate the power of the getML feature learning algorithms. It is very unlikely to find these features using manual, trial-and-error based methods. The general structure of features found using such methods might be similar, but you would have had to put in much more effort while getting worse results.

When browsing through the remaining features, you will notice that some are columns directly taken from the original table, such as *Light* and *CO2*. But these columns are less correlated and less important than the features generated with the relational model based on self-join and upper time stamps.

Because getML uses a feature learning approach, the concept of feature importances can also be carried over to the individual columns.

In [ ]:
names, importances = pipe.columns.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color="#6829c2")

plt.title("column importances")
plt.grid(True)
plt.xlabel("columns")
plt.ylabel("importance")
plt.xticks(rotation="vertical")

plt.show()

In [ ]:
pipe.columns

As we can see, about 80% of the predictive power comes from the *Light* column. The second most important column is the *Temperature* column, we contributes about 7% of the predictive power.

 ### 2.5 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.


In [ ]:
# Creates a folder named occupancy_pipeline containing
# the SQL code.
pipe.features.to_sql().save("occupancy_pipeline")

In [ ]:
pipe.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("occupancy_spark")

## 3. Conclusion

This tutorial demonstrates that relational learning is a powerful tool for time series. We able to outperform the benchmarks for a scientific paper on a simple public domain time series data set using relatively little effort.

If you want to learn more about getML, check out the [official documentation](https://getml.com/product).

# Next Steps

This tutorial went through the basics of applying getML to multirvariate time series and hyperparameter tuning.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.